# Clean up an expression matrix from a dataset in GEO

In this notebook, we'll download dataset [GSE70630](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE70630) from GEO by [Tirosh et al. (2016)](https://pubmed.ncbi.nlm.nih.gov/27806376/) (Specifically, the supplemental file "GSE70630_OG_processed_data_v2"). We'll select just two tumors from this dataset, transform its expression values into log(TPM+1) and output the data in the form of a TSV file. This output file will serve as the toy dataset for demonstrating how to run the CHARTS pipeline.  

In [2]:
from optparse import OptionParser
import pandas as pd
import json
import numpy as np
import h5py
from os.path import join
from anndata import AnnData
import scanpy as sc
from collections import defaultdict

root = '/Users/matthewbernstein/Development/charts/raw_data2' # Location of downloaded from GEO
df = pd.read_csv(
    join(root, 'GSE70630_OG_processed_data_v2.txt'),
    sep='\t',
    index_col=0
)
df

Loading data...


,MGH36_P6_A12,MGH36_P6_H09,MGH53_P4_G04,MGH36_P10_G12,MGH53_P2_H12,MGH53_P4_D10,MGH53_P4_D01,MGH36_P6_B07,MGH36_P10_B12,MGH53_P2_G11,...,93_P10_H06,93_P8_B12,93_P8_D09,93_P9_D11,93_P10_G08,93_P8_H06,93_P9_C07,93_P8_A12,93_P8_C01,93_P9_F06
'A1BG',0.00000,0.0000,0.00000,0.00000,0.000000,0.00000,0.0000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000
'A1BG-AS1',0.00000,0.0000,0.00000,0.00000,0.000000,0.00000,0.0000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000
'A1CF',0.00000,0.0000,0.00000,0.00000,0.021480,0.00000,0.0000,0.000000,0.527070,0.000000,...,0.00000,0.000000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000
'A2M',5.70560,4.4370,8.02760,5.62880,0.000000,3.33670,8.7811,8.327100,7.426200,9.046200,...,0.00000,0.000000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000
'A2M-AS1',0.00000,0.0000,4.53470,0.00000,0.000000,0.00000,0.0000,3.336100,0.000000,0.631340,...,0.00000,0.000000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
'ZYG11A',0.35163,0.0000,0.70752,0.00000,0.221570,0.00000,1.2910,0.932060,0.401630,0.618240,...,0.00000,0.000000,0.0000,0.63227,0.438290,0.00000,0.20163,0.270230,0.00000,0.37740
'ZYG11B',0.00000,2.2185,0.84398,1.22590,0.144050,0.67987,4.0467,3.739500,0.000000,0.096262,...,0.29983,0.064883,0.0000,0.18396,0.247320,0.27262,4.06500,0.049631,0.42223,0.47923
'ZYX',1.36180,3.2621,0.00000,0.00000,4.980900,0.00000,0.0000,0.000000,0.680770,2.255800,...,3.63040,3.220600,0.0000,0.00000,1.912600,0.00000,0.00000,1.348200,3.45900,3.19980
'ZZEF1',1.59980,0.0000,0.00000,0.26903,0.035624,0.00000,0.0000,0.066261,0.084064,0.000000,...,1.25280,0.000000,0.0000,2.39510,0.063503,0.97893,0.00000,1.741100,1.29040,5.25520


### Take transpose and drop NaN values from matrix

In [3]:
df = df.transpose()
df = df.dropna(axis=1) # For some reason, a couple dozen genes have NaN values
df

,'A1BG','A1BG-AS1','A1CF','A2M','A2M-AS1','A2ML1','A2MP1','A4GALT','A4GNT','AA06',...,'ZWILCH','ZWINT','ZXDA','ZXDB','ZXDC','ZYG11A','ZYG11B','ZYX','ZZEF1','ZZZ3'
MGH36_P6_A12,0.0,0.0,0.00000,5.7056,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.00000,0.53506,0.35163,0.000000,1.3618,1.599800,0.00000
MGH36_P6_H09,0.0,0.0,0.00000,4.4370,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.00000,0.14535,0.00000,2.218500,3.2621,0.000000,0.00000
MGH53_P4_G04,0.0,0.0,0.00000,8.0276,4.5347,0.32077,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.00000,2.77740,0.70752,0.843980,0.0000,0.000000,0.00000
MGH36_P10_G12,0.0,0.0,0.00000,5.6288,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.00000,1.11700,0.00000,1.225900,0.0000,0.269030,0.00000
MGH53_P2_H12,0.0,0.0,0.02148,0.0000,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.00000,0.12168,0.22157,0.144050,4.9809,0.035624,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93_P8_H06,0.0,0.0,0.00000,0.0000,0.0000,0.21785,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.00000,3.81660,0.00000,0.272620,0.0000,0.978930,3.51530
93_P9_C07,0.0,0.0,0.00000,0.0000,0.0000,0.42223,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.00000,0.17632,0.20163,4.065000,0.0000,0.000000,0.00000
93_P8_A12,0.0,0.0,0.00000,0.0000,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,2.147,0.00000,0.14666,0.27023,0.049631,1.3482,1.741100,0.00000
93_P8_C01,0.0,0.0,0.00000,0.0000,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.00000,0.72509,0.00000,0.422230,3.4590,1.290400,0.00000


### Map each tumor to its cell ID's

In [5]:
tumor_to_cells = defaultdict(lambda: [])
for cell_id in df.index:
    raw_tum = cell_id.split('_')[0]
    if 'MGH' in raw_tum:
        tumor = raw_tum
    else:
        tumor = 'MGH{}'.format(raw_tum)
    tumor_to_cells[tumor].append(cell_id)
    
# Print the number of cells from each tumor
print({k: len(v) for k,v in tumor_to_cells.items()})

{'MGH36': 788, 'MGH53': 861, 'MGH54': 1225, 'MGH60': 430, 'MGH97': 598, 'MGH93': 445}


### Select data from tumors MGH36 and MGH53

In [8]:
select_tumors = ['MGH36', 'MGH53']

cells_tum1 = tumor_to_cells['MGH36']
df_tumor1 = df.loc[cells_tum1]
df_tumor1['tumor'] = ['MGH36' for i in range(len(cells_tum1))]


cells_tum2 = tumor_to_cells['MGH53']
df_tumor2 = df.loc[cells_tum2]
df_tumor2['tumor'] = ['MGH53' for i in range(len(cells_tum2))]

df_selected = pd.concat([df_tumor1, df_tumor2])
df_selected

,'A1BG','A1BG-AS1','A1CF','A2M','A2M-AS1','A2ML1','A2MP1','A4GALT','A4GNT','AA06',...,'ZWINT','ZXDA','ZXDB','ZXDC','ZYG11A','ZYG11B','ZYX','ZZEF1','ZZZ3',tumor
MGH36_P6_A12,0.0,0.0000,0.000000,5.70560,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0000,0.53506,0.35163,0.00000,1.36180,1.599800,0.0,MGH36
MGH36_P6_H09,0.0,0.0000,0.000000,4.43700,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0000,0.14535,0.00000,2.21850,3.26210,0.000000,0.0,MGH36
MGH36_P10_G12,0.0,0.0000,0.000000,5.62880,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0000,1.11700,0.00000,1.22590,0.00000,0.269030,0.0,MGH36
MGH36_P6_B07,0.0,0.0000,0.000000,8.32710,3.3361,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0000,0.46467,0.93206,3.73950,0.00000,0.066261,0.0,MGH36
MGH36_P10_B12,0.0,0.0000,0.527070,7.42620,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.47923,0.0000,0.24245,0.40163,0.00000,0.68077,0.084064,0.0,MGH36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MGH53_P4_G08,0.0,0.0000,0.000000,0.00000,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.00000,4.1648,0.82375,0.30685,0.16993,4.66630,0.000000,0.0,MGH53
MGH53_P4_G09,0.0,0.0000,0.041243,0.43296,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0000,0.12829,0.17377,0.45418,0.00000,0.000000,0.0,MGH53
MGH53_P4_H10,0.0,4.0587,0.000000,0.00000,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0000,0.21412,0.24732,0.29160,0.00000,0.000000,0.0,MGH53
MGH53_P4_H11,0.0,0.0000,0.000000,0.31615,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0000,0.29160,0.36065,0.44042,0.00000,0.000000,0.0,MGH53


### Compute log(TPM+1) expression values

In [14]:
# Compute log(TPM+1) 
X = np.array(df_selected[df_selected.columns[:-1]], dtype=np.float32)
X = 10*(np.power(2, X)-1)
ad = AnnData(
    X=X
)
sc.pp.log1p(ad)
log1_tpm = ad.X

df_out = pd.DataFrame(
    data=X,
    columns=df_selected.columns[:-1],
    index=df_selected.index
)
df_out['tumor'] = list(df_selected['tumor'])
df_out

,'A1BG','A1BG-AS1','A1CF','A2M','A2M-AS1','A2ML1','A2MP1','A4GALT','A4GNT','AA06',...,'ZWINT','ZXDA','ZXDB','ZXDC','ZYG11A','ZYG11B','ZYX','ZZEF1','ZZZ3',tumor
MGH36_P6_A12,0.0,0.000000,0.000000,6.240009,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,1.702933,1.324423,0.000000,2.815442,3.059183,0.0,MGH36
MGH36_P6_H09,0.0,0.000000,0.000000,5.335641,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.722701,0.000000,3.625432,4.465193,0.000000,0.0,MGH36
MGH36_P10_G12,0.0,0.000000,0.000000,6.185816,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,2.540779,0.000000,2.666543,0.000000,1.115133,0.0,MGH36
MGH36_P6_B07,0.0,0.000000,0.000000,8.071685,4.521649,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,1.568619,2.310552,4.824850,0.000000,0.385260,0.0,MGH36
MGH36_P10_B12,0.0,0.000000,1.688248,7.444788,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,1.597364,0.000000,1.040277,1.437462,0.000000,1.950180,0.470003,0.0,MGH36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MGH53_P4_G08,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,5.137924,2.163324,1.214924,0.810948,5.500930,0.000000,0.0,MGH53
MGH53_P4_G09,0.0,0.000000,0.254643,1.504079,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.657507,0.824185,1.547571,0.000000,0.000000,0.0,MGH53
MGH53_P4_H10,0.0,5.060341,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.955497,1.054312,1.175564,0.000000,0.000000,0.0,MGH53
MGH53_P4_H11,0.0,0.000000,0.000000,1.238385,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,1.175564,1.345484,1.519512,0.000000,0.000000,0.0,MGH53


### Output expression matrix

In [16]:
df_out.to_csv('~/Desktop/GSE70630_MGH36_MGH53.tsv', sep='\t')